In [5]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
! source ~/.bashrc
import numpy as np
import matplotlib.pyplot as plt
from spp.convex_sets import Singleton, Polyhedron, CartesianProduct
from spp.convex_functions import SquaredTwoNorm
from spp.pwa_systems import PieceWiseAffineSystem, ShortestPathRegulator

/bin/sh: 1: source: not found


ModuleNotFoundError: No module named 'pydrake.all'

In [ ]:
# initial state
z1 = np.array([-3.5, .5, 0, 0])
q1 = z1[:2]

# target set
zK = np.array([3.5, 6.5, 0, 0])
qK = zK[:2]
Z = Singleton(zK)

# time horizon
K = 30

# cost matrices
q_dot_cost = .2 ** .5
Q = np.diag([0, 0, q_dot_cost, q_dot_cost])
R = np.eye(2)
S = Q # ininfluential
cost_matrices = (Q, R, S)

In [3]:
# configuration bounds
Dq = [
    Polyhedron.from_bounds([-4, 0], [3, 1]),
    Polyhedron.from_bounds([-6, 1], [-5, 3]),
    Polyhedron.from_bounds([4, 1], [5, 2]),
    Polyhedron.from_bounds([-4, 3], [4, 4]),
    Polyhedron.from_bounds([-5, 5], [-4, 6]),
    Polyhedron.from_bounds([5, 4], [6, 6]),
    Polyhedron.from_bounds([-3, 6], [4, 7])
]

# velocity bounds
qdot_max = np.ones(2) * 1
qdot_min = - qdot_max
Dqdot = Polyhedron.from_bounds(qdot_min, qdot_max)

# control bounds
u_max = np.ones(2) * 1
u_min = - u_max
Du = Polyhedron.from_bounds(u_min, u_max)

# pwa domains
domains = [CartesianProduct((Dqi, Dqdot, Du)) for Dqi in Dq]

NameError: name 'Polyhedron' is not defined

In [ ]:
# dynamics
A = np.array([
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])
B = np.vstack((np.zeros((2, 2)), np.eye(2)))
Bred = B / 10
c = np.zeros(4)
dynamics = [(A, Bred, c) if i in [1, 5] else (A, B, c) for i in range(len(domains))]

# pieceiwse affine system
pwa = PieceWiseAffineSystem(dynamics, domains)

In [ ]:
def plot_terrain(q=None, u=None):
    plt.rc('axes', axisbelow=True)
    plt.gca().set_aspect('equal')

    for i, Dqi in enumerate(Dq):
        color = 'lightcoral' if i in [1, 5] else 'lightcyan'
        Dqi.plot(facecolor=color)
        
    plt.scatter(*q1, s=300, c='g', marker='+', zorder=2)
    plt.scatter(*qK, s=300, c='g', marker='x', zorder=2)
    
    if q is not None:
        plt.plot(*q.T, c='k', marker='o', markeredgecolor='k', markerfacecolor='w')
        
        if u is not None:
            for t, ut in enumerate(u):
                plt.arrow(*q[t], *ut, color='b', head_starts_at_zero=0, head_width=.15, head_length=.3)
                
    plt.xlabel(r'$q_1, w_1$')
    plt.ylabel(r'$q_2, w_2$')
    plt.grid(1)

In [ ]:
# plot solution
plt.figure(figsize=(4, 3))
plot_terrain(q, u)
plt.xticks(range(-6, 7))

plt.savefig('footstep_spp.pdf', bbox_inches='tight')

# Baseline formulation

In [ ]:
from pydrake.all import MathematicalProgram, MosekSolver, le, ge, eq，SolverOptions

def solve_baseline(relaxation=0):
    
    # initialize program
    prog = MathematicalProgram()
    
    # continuous decision variables
    z = prog.NewContinuousVariables(K, 4)
    u = prog.NewContinuousVariables(K - 1, 2)
    q = z[:, :2]
    qdot = z[:, 2:]
    
    # indicator variables
    if relaxation:
        b = prog.NewContinuousVariables(K - 1, len(Dq))
        prog.AddLinearConstraint(ge(b.flatten(), 0))
    else:
        b = prog.NewBinaryVariables(K - 1, len(Dq))
        
    # slack variables for the cost function
    s = prog.NewContinuousVariables(K - 1, len(Dq))
    prog.AddLinearConstraint(ge(s.flatten(), 0))
    prog.AddLinearCost(sum(sum(s)))
    
    # initial and terminal conditions
    prog.AddLinearConstraint(eq(z[0], z1))
    prog.AddLinearConstraint(eq(z[-1], zK))
    
    # containers for copies of state and the controls
    z_aux = []
    u_aux = []
    
    # loop over time
    for k in range(K - 1):
        
        # auxiliary copies of state and the controls
        Z = prog.NewContinuousVariables(len(Dq), 4)
        U = prog.NewContinuousVariables(len(Dq), 2)
        Znext = prog.NewContinuousVariables(len(Dq), 4)
        Q = Z[:, :2]
        Qdot = Z[:, 2:]
        z_aux.append(Z)
        u_aux.append(U)
        
        # loop over modes of the pwa system
        for i, Dqi in enumerate(Dq):
            
            # cone constraint for the perspective of the cost
            prog.AddRotatedLorentzConeConstraint(np.concatenate((
                [s[k, i]],
                [b[k, i]],
                U[i],
                q_dot_cost * Qdot[i]
                )))
            
            # state and input bounds
            prog.AddLinearConstraint(le(Dqi.C.dot(Q[i]), Dqi.d * b[k, i]))
            prog.AddLinearConstraint(le(U[i], u_max * b[k, i]))
            prog.AddLinearConstraint(ge(U[i], - u_max * b[k, i]))
            prog.AddLinearConstraint(le(Qdot[i], qdot_max * b[k, i]))
            prog.AddLinearConstraint(ge(Qdot[i], - qdot_max * b[k, i]))
            
            # pwa dynamics
            Ai, Bi, ci = pwa.dynamics[i]
            prog.AddLinearConstraint(eq(Ai.dot(Z[i]) + Bi.dot(U[i]) + ci, Znext[i]))
            
            # reconstruct auxiliary variables
            prog.AddLinearConstraint(eq(sum(Z), z[k]))
            prog.AddLinearConstraint(eq(sum(U), u[k]))
            prog.AddLinearConstraint(eq(sum(Znext), z[k + 1]))
            prog.AddLinearConstraint(sum(b[k]) == 1)
            
    # solve optimization
    solver = MosekSolver()
    result = solver.Solve(prog)
    print('Cost:', result.get_optimal_cost())
    print('Solve time:', result.get_solver_details().optimizer_time)
    
    # get optimal solution
    z_opt = result.GetSolution(z)
    u_opt = result.GetSolution(u)
    b_opt = result.GetSolution(b)
    z_aux_opt = [result.GetSolution(zk) for zk in z_aux]
    u_aux_opt = [result.GetSolution(uk) for uk in u_aux]
    
    return z_opt, u_opt, b_opt, z_aux_opt, u_aux_opt

In [ ]:
# solve the stepping stone problem using the perspective formulation
relaxation = 1
z, u, b, z_aux, u_aux = solve_baseline(relaxation)
q = z[:, :2]
qdot = z[:, 2:]

In [ ]:
# plot optimal solution
plt.figure(figsize=(4, 3))
plot_terrain(q, u)
plt.xticks(range(-6, 7))

# plot transparent triangles
if relaxation:
    for k in range(K - 1):
        for i, Dqi in enumerate(Dq):
            if not np.isclose(b[k][i], 0):
                plt.scatter(
                    *z_aux[k][i][:2] / b[k][i],
                    alpha=b[k][i],
                    marker='^', edgecolor='k', facecolor='w', zorder=2
                )
# plt.savefig('footstep_pf.pdf', bbox_inches='tight')